In [2]:
import pandas as pd
import pytz

# File paths
input_file = "C:\\Users\\Nathanael\\Dropbox\\Centrale_etudes\\CSEC powerdot\\energy_metering.csv"
output_file = "C:\\Users\\Nathanael\\Dropbox\\Centrale_etudes\\CSEC powerdot\\energy_metering_utc.csv"

# Load the dataset
df = pd.read_csv(input_file)

# Convert the meter_timestamp column to datetime format
df["meter_timestamp"] = pd.to_datetime(df["meter_timestamp"])

# Define timezones
paris_tz = pytz.timezone("Europe/Paris")
utc_tz = pytz.utc

# Localize to Europe/Paris, handling DST ambiguity by setting ambiguous times to NaT (Not-a-Time)
df["meter_timestamp"] = df["meter_timestamp"].dt.tz_localize(paris_tz, ambiguous="NaT", nonexistent="shift_forward")

# Drop NaT values (ambiguous timestamps that could not be resolved)
df.dropna(subset=["meter_timestamp"], inplace=True)

# Convert to UTC
df["meter_timestamp"] = df["meter_timestamp"].dt.tz_convert(utc_tz)

# Save the new dataset
df.to_csv(output_file, index=False)

print(f"Conversion complete. File saved as: {output_file}")



Conversion complete. File saved as: C:\Users\Nathanael\Dropbox\Centrale_etudes\CSEC powerdot\energy_metering_utc.csv


In [1]:
import pandas as pd

# File paths
energy_file = "C:\\Users\\Nathanael\\Dropbox\\Centrale_etudes\\CSEC powerdot\\energy_metering_utc.csv"
charging_file = "C:\\Users\\Nathanael\\Dropbox\\Centrale_etudes\\CSEC powerdot\\max_consumption_per_location.csv"
output_file = "C:\\Users\\Nathanael\\Dropbox\\Centrale_etudes\\CSEC powerdot\\leakages.csv"

# Load datasets
df_energy = pd.read_csv(energy_file)
df_charging = pd.read_csv(charging_file)

# Convert timestamps to datetime format
df_energy["meter_timestamp"] = pd.to_datetime(df_energy["meter_timestamp"], utc=True)  # Ensure UTC timezone
df_energy["hour"] = df_energy["meter_timestamp"].dt.floor("H")  # Round to full hour in datetime format

df_charging["day"] = pd.to_datetime(df_charging["day"])  # Ensure day is in datetime format

# Convert df_charging hour column into a full datetime format (combine day and hour)
df_charging["hour"] = pd.to_datetime(df_charging["day"]) + pd.to_timedelta(df_charging["hour"], unit="h")

# Ensure both datasets have UTC timezone
df_charging["hour"] = df_charging["hour"].dt.tz_localize("UTC")  # Ensure it's in UTC

# Merge energy data with charging data on meter_id and hour
df_merged = df_energy.merge(
    df_charging,
    how="left",
    on=["meter_id", "hour"]  # Now both columns are in datetime format
)

# Identify energy leakages:
# - There is power consumption (power_w > 0)
# - No cars were charging (max_consumption == 0 or NaN)
df_leakages = df_merged[(df_merged["power_w"] > 0) & ((df_merged["max_consumption"].isna()) | (df_merged["max_consumption"] == 0))]

# Select only the necessary columns: meter_id and hour
df_leakages = df_leakages[["meter_id", "hour"]].drop_duplicates()

# Save to CSV
df_leakages.to_csv(output_file, index=False)

print(f"Energy leakage detection complete. Leakages saved in: {output_file}")



C:\Users\Nathanael\AppData\Local\Temp\ipykernel_21328\1863865742.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_energy["hour"] = df_energy["meter_timestamp"].dt.floor("H")  # Round to full hour in datetime format


Energy leakage detection complete. Leakages saved in: C:\Users\Nathanael\Dropbox\Centrale_etudes\CSEC powerdot\leakages.csv
